PS D:\MS Purdue\1\ECE570> python --version

Python 3.9.0

PS D:\MS Purdue\1\ECE570> py -3.11 --version

Python 3.11.9

py -3.11 -m pip list

py -3.11 -m pip install <package_name>

In [1]:
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import os
import torch.optim as optim


# Print the current working directory
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

c:\Users\alanc\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current directory: d:\MS Purdue\1\ECE570\Project\1stAttempt(without fancy, use pretrained)


In [2]:
transform = transforms.Compose([
    # Resize to Tiny ImageNet's image size
    transforms.Resize((128, 128)),  
    transforms.ToTensor(),
    # Normalization
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalization
])

# Path to dataset
data_dir = '../tiny-imagenet-200Class-500Data/'

# Load training and validation data
train_dataset = datasets.ImageFolder(root=data_dir + 'train', transform=transform)
val_dataset = datasets.ImageFolder(root=data_dir + 'val', transform=transform)


# DataLoader for batching
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Check class-to-index mapping
print(train_dataset.class_to_idx) # Class labels
# Check class-to-index mapping and dataset size
print(train_dataset.class_to_idx)  # Class labels

print(f"Train subset size: {len(train_dataset)}")
print(f"Validation subset size: {len(val_dataset)}")

# Example: Iterate through the train_loader to test
for images, labels in train_loader:
    print(images.shape, labels.shape)
    break

{'n01443537': 0, 'n01629819': 1, 'n01641577': 2, 'n01644900': 3, 'n01698640': 4, 'n01742172': 5, 'n01768244': 6, 'n01770393': 7, 'n01774384': 8, 'n01774750': 9, 'n01784675': 10, 'n01855672': 11, 'n01882714': 12, 'n01910747': 13, 'n01917289': 14, 'n01944390': 15, 'n01945685': 16, 'n01950731': 17, 'n01983481': 18, 'n01984695': 19, 'n02002724': 20, 'n02056570': 21, 'n02058221': 22, 'n02074367': 23, 'n02085620': 24, 'n02094433': 25, 'n02099601': 26, 'n02099712': 27, 'n02106662': 28, 'n02113799': 29, 'n02123045': 30, 'n02123394': 31, 'n02124075': 32, 'n02125311': 33, 'n02129165': 34, 'n02132136': 35, 'n02165456': 36, 'n02190166': 37, 'n02206856': 38, 'n02226429': 39, 'n02231487': 40, 'n02233338': 41, 'n02236044': 42, 'n02268443': 43, 'n02279972': 44, 'n02281406': 45, 'n02321529': 46, 'n02364673': 47, 'n02395406': 48, 'n02403003': 49, 'n02410509': 50, 'n02415577': 51, 'n02423022': 52, 'n02437312': 53, 'n02480495': 54, 'n02481823': 55, 'n02486410': 56, 'n02504458': 57, 'n02509815': 58, 'n0266

In [5]:
import torch.nn as nn


# Initialize the model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available

# Import the specific functions from the vanillanet.py file
from vanillanet_NN import vanillanet_5, vanillanet_6, vanillanet_9

pretrained_weights_path = 'vanillanet_5.pth'  # Update this with the actual path

# Create an instance of the VanillaNet model (VanillaNet-9 in this case)
model = vanillanet_5(pretrained=True, num_classes=200).to(device)  
model.fc = nn.Linear(model.fc.in_features, 200)

pretrained_dict = torch.load(pretrained_weights_path, map_location=device)

# Check if the state_dict contains nested keys like 'model' or 'model_ema'
if 'model' in pretrained_dict:
    pretrained_dict = pretrained_dict['model']  # Extract the actual model weights
    print("1111111111\n1111111111111\n")
elif 'model_ema' in pretrained_dict:
    pretrained_dict = pretrained_dict['model_ema']
    print("222222222222\n2222222222222\n")

# Print model architecture to verify
print(model)


AttributeError: 'VanillaNet' object has no attribute 'fc'

In [4]:
import torch.optim as optim
import matplotlib.pyplot as plt  # For plotting

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-3)  # Adam optimizer. adjust weight decay for overfitting

# Initialize lists to store metrics
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

num_epochs = 100  # Number of epochs
patience = 5  # Patience for early stopping
best_val_loss = float('inf')  # Initialize best validation loss as infinity
epochs_without_improvement = 0  # Counter for how many epochs passed without improvement

early_stop = False

# Initialize the learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min',      # Reduce when the quantity monitored has stopped decreasing
                                                 factor=0.5,     # Factor by which the learning rate will be reduced
                                                 patience=1,     # Number of epochs with no improvement after which learning rate will be reduced
                                                 verbose=True)   # Print a message when the learning rate is reduced

for epoch in range(num_epochs):
    if early_stop:
        print("Early stopping triggered.")
        break
    
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to device

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Update running loss
        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # Calculate average training loss and accuracy
    avg_train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # Validation loop
    model.eval()  # Set model to evaluation mode
    correct_val = 0
    total_val = 0
    val_loss = 0.0

    with torch.no_grad():  # No need to calculate gradients during validation
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    # Calculate average validation loss and accuracy
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val

    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

    # Step the scheduler
    scheduler.step(avg_val_loss)  # Update the learning rate based on validation loss


    # Print epoch statistics
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
    print(f'Validation Accuracy: {val_accuracy:.2f}%, Validation Loss: {avg_val_loss:.4f}')

    # Early stopping logic
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss  # Update the best validation loss
        epochs_without_improvement = 0  # Reset the counter if we have improvement
        best_model_wts = model.state_dict()  # Optionally save the best model
    else:
        epochs_without_improvement += 1  # Increment the counter

    if epochs_without_improvement >= patience:
        print(f'Early stopping after {epoch+1} epochs without improvement.')
        early_stop = True
        # load the best model weights
        model.load_state_dict(best_model_wts)

print('Training complete!')

# Plotting the results
epochs = range(1, len(train_losses) + 1)

plt.figure(figsize=(12, 5))

# Plot training & validation loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, 'bo-', label='Train Loss')
plt.plot(epochs, val_losses, 'ro-', label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training & validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, 'bo-', label='Train Accuracy')
plt.plot(epochs, val_accuracies, 'ro-', label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.show()

# Save the model with the best performance
if best_model_wts is not None:
    model.load_state_dict(best_model_wts)  # Ensure we load the best model weights before saving
    torch.save(model.state_dict(), 'best_model.pth')  # Save model to a file
    print("Best model saved as 'best_model.pth'")
else:
    print("No improvement in validation loss, model not saved.")

Epoch [1/100], Train Loss: 4.9807, Train Accuracy: 2.53%
Validation Accuracy: 2.88%, Validation Loss: 5.1012
Epoch [2/100], Train Loss: 4.7197, Train Accuracy: 4.70%
Validation Accuracy: 3.85%, Validation Loss: 4.8588
Epoch [3/100], Train Loss: 4.5411, Train Accuracy: 6.65%
Validation Accuracy: 6.16%, Validation Loss: 4.5871


: 

: 